# Secp256k1

Constants from https://en.bitcoin.it/wiki/Secp256k1

In [6]:
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
F = FiniteField(p)

In [7]:
a = 0
b = 7
E = EllipticCurve(F, [a, b])

In [8]:
G_x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
h = 1 # cofactor
G = B = E.lift_x(G_x)

In [9]:
assert(G.order() == 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)

In [10]:
assert(E.order() / G.order() == h)

# Key generation

In [11]:
N = FiniteField(G.order())

In [12]:
def keypair (x = None):
    if x is None:
        x = int(N.random_element(1, N.order() - 1))
    P = x * G
    return (x, P)

# ECDSA

In [13]:
import hashlib
def sign (m, d):
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    k = int(N.random_element(1, N.order() - 1))
    R = k * G # (k, r) is like a one-time key pair
    r = int(R.xy()[0]) % N.order()
    assert(r % N.order() != 0)
    s = (e + d*r)/k % N.order()
    return [r,s]
    

In [15]:
def verify (sig, m, P):
    r, s = sig
    assert(0 < r < N.order())
    assert(0 < s < N.order())
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    u1 = e / s % N.order()
    u2 = r / s % N.order()
    R = u1 * G + u2 * P
    v = int(R.xy()[0]) % N.order()
    return v == r

# ID based signatures

In [12]:
id = int(hashlib.sha256(b"Emil bayes").hexdigest(), 16) % N.order()

In [13]:
Q = id * P

In [14]:
z = id * x % N.order()

In [15]:
sig2 = sign(b"Hello world", z)

In [16]:
verify(sig2, b"Hello world", Q)

True